In [1]:
# definer
import re
import pandas as pd
import numpy as np
# import textmining


In [2]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [3]:
MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [4]:
df = pd.read_csv('./data/train.tsv', sep='\t', header=0)

In [5]:
Nd = np.zeros(5)
print(Nd[3])

0.0


In [6]:
# reading data
# df = pd.read_excel('dataset.xlsx')
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['Sentiment'])))

('Shape of dataset ', (156060, 4))
Index([u'PhraseId', u'SentenceId', u'Phrase', u'Sentiment'], dtype='object')
('No. of unique classes', 5)


In [7]:
macronum=sorted(set(df['Sentiment']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['Sentiment']=df['Sentiment'].apply(fun)

In [8]:
texts = []
labels = []
# tdm = textmining.TermDocumentMatrix()
for idx in df['Sentiment']:
    labels.append(idx)

for idx in range(df.Phrase.shape[0]):
#     print(df.Phrase[idx])
    text = df.Phrase[idx]
    strr = clean_str(text)
    Nd[labels[idx]] += len(strr.split())
    texts.append(strr)
#     tdm.add_doc(clean_str(text))



In [9]:
pLabel0 = labels.count(0) * 1.0 / len(labels)
pLabel1 = labels.count(1) * 1.0 / len(labels)
pLabel2 = labels.count(2) * 1.0 / len(labels)
pLabel3 = labels.count(3) * 1.0 / len(labels)
pLabel4 = labels.count(4) * 1.0 / len(labels)
print(pLabel0)
print(pLabel1)
print(pLabel2)
print(pLabel3)
print(pLabel4)


0.0453159041394
0.174759707805
0.50994489299
0.210989363065
0.0589901320005


In [10]:
# tdm

In [11]:
# cntr = 0
# textLabel = []
# trainData = []
# for row in tdm.rows(cutoff=1):
#     cntr = cntr + 1
#     if cntr == 1:
#         textLabel = row
#     else:
#         trainData.append(row)

# print(len(trainData) , len(trainData[0]))

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(texts)
df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names())
print(df.shape)


(156060, 15241)


In [13]:
## AFTER Df being the document term frequency

# P(label | text) === P(text | label) 
# CAlc P(text | label) for the whole VOCAB 
# So a matrix/dictionary of a['word'] = probability of 5 labels SO V*5 matrix

In [14]:
alp = 0.000000001

In [15]:
print(Nd)

[ 85270. 247523. 411176. 277216.  98250.]


In [16]:
xtf = np.zeros([15241,5])
print(xtf.shape)

(15241, 5)


In [17]:
lablls = np.asarray(labels)
print(lablls == 3)

[False False False ...  True False False]


In [18]:
j = 0
for column in df:
    labl = 0
    
    k = np.array(df[column])
    while (labl < 5):
        prrod = np.sum(np.dot(k,lablls==labl))
        """
        sum of term of word defined by column for labl
        and then
        that sum (prrod) divided by all sum of word for th given label and then alp for
        smoothening a hyper parameter.
        """
        ans = (prrod + alp) * (1.0/( Nd[labl] + alp*k.shape[0] ))
        xtf[j][labl] = ans
        labl += 1
    j = j + 1

print(j)

15241


In [19]:
sample_text = "This is a good pen"

sample_text = clean_str(sample_text)
header_list = list(df)


In [20]:
import sys
sys.stdout = open('./out.csv', 'w')

df = pd.read_csv('./data/test.tsv', sep='\t', header=0)
for idx in range(df.Phrase.shape[0]):
    sample_text = df.Phrase[idx]
    sample_text = clean_str(sample_text)
    x = len(sample_text.split())
    l0 = 1
    l1 = 1
    l2 = 1
    l3 = 1
    l4 = 1
    for i in range(x):
        if header_list.count(sample_text.split()[i]) > 0 :
            l0 *= xtf[header_list.index(sample_text.split()[i])][0] * pLabel0
            l1 *= xtf[header_list.index(sample_text.split()[i])][1] * pLabel1
            l2 *= xtf[header_list.index(sample_text.split()[i])][2] * pLabel2
            l3 *= xtf[header_list.index(sample_text.split()[i])][3] * pLabel3
            l4 *= xtf[header_list.index(sample_text.split()[i])][4] * pLabel4
    lp = np.zeros(5)
    lp[0] = l0/ (l0 + l1 + l2 + l3 + l4)    
    lp[1] = l1/ (l0 + l1 + l2 + l3 + l4)    
    lp[2] = l2/ (l0 + l1 + l2 + l3 + l4)    
    lp[3] = l3/ (l0 + l1 + l2 + l3 + l4)    
    lp[4] = l4/ (l0 + l1 + l2 + l3 + l4)    
    print(lp.argmax())

# print("So the guessed rating is ")
